并发是指一次处理多件事；并行是指一次做多件事。

asyncio 包使用事件循环驱动的协程实现并发。

Python 没有提供终止线程的 API，若想关闭线程，必须给线程发送消息。

In [1]:
# spinner_thread.py
import threading
import itertools
import time


def spin(msg, done):  # 在单独的线程中运行
    for char in itertools.cycle('|/-\\'):  # 无限循环
        status = char + ' ' + msg
        print(status, flush=True, end='\r')
        if done.wait(.1):  # 退出循环
            break
    print(' ' * len(status), end='\r')


def slow_function():  # 假设是耗时的计算
    # 假装等待 I/O
    time.sleep(3)  # 阻塞主线程，释放 GIL，创建从属线程
    return 42


def supervisor():  # 设置从属线程
    done = threading.Event()
    spinner = threading.Thread(target=spin,
                               args=('thinking!', done))
    print('spinner object:', spinner)  # 显示线程对象
    spinner.start()  # 启动线程
    result = slow_function()  # 阻塞主线程
    done.set()  # 终止 spin 函数的无限循环
    spinner.join()  # 等线程结束
    return result


def main():
    result = supervisor()  # 运行
    print('Answer:', result)


if __name__ == '__main__':
    main()

spinner object: <Thread(Thread-6, initial)>
Answer: 42 


如果协程需要在一段时间内什么也不做，应该使用 await asycio.sleep(DELAY)

In [2]:
#spinner_asyncio.py
import asyncio
import itertools


async def spin(msg):
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg
        print(status, flush=True, end='\r')
        try:
            await asyncio.sleep(.1)  # 不阻塞事件循环
        except asyncio.CancelledError:  # 取消请求，退出循环
            break
    print(' ' * len(status), end='\r')


async def slow_function():  # 协程
    # 假装等待 I/O
    await asyncio.sleep(3)  # 把控制权交给主循环，休眠结束后恢复协程
    return 42


async def supervisor():  # 协程
    spinner = asyncio.create_task(spin('thinking!'))  # 排定，包装协程
    print('spinner object:', spinner)  # 显示 Task 对象
    result = await slow_function()  # 驱动
    spinner.cancel()  # 取消，抛出异常（协程可以捕获、延迟取消、拒绝取消）
    return result


# def main():
#     result = asyncio.run(supervisor())  # 等待协程运行完毕
#     print('Answer:', result)

async def main():
    result = await supervisor()
    print('Answer:', result)

if __name__ == '__main__':
    await main()

spinner object: <Task pending name='Task-2' coro=<spin() running at <ipython-input-2-17707867df98>:6>>
Answer: 42!


spinner_thread.py 和 spinner_asyncio.py 中的 supervisor 实现之间的主要区别：
- asyncio.Task 对象差不多与 threading.Thread 对象等效
- Task 对象用于驱动协程，Thread 对象用于调用可调用的对象
- Task 对象不由自己手动实例化，而是通过把协程传给 asyncio.async(...) 函数或 loop.create_task(...) 方法获取
- 获取的 Task 对象已经排定了运行时间（由 asyncio.async 函数排定）；Thread 实例必须调用 start 方法，明确告知让它运行
- spinner_thread.py 中，slow_function 函数是普通的函数，直接由线程调用；spinner_asyncio.py 中，slow_function 函数是协程，由 yield from 驱动
- 没有 API 能从外部终止线程。如果想终止任务，可以使用 Task.cancel() 实例方法，在协程内部抛出 CancelledError 异常；协程可以在暂停的 yield 处捕获这个异常，处理终止请求。

线程
- 调度程序任何时候都能中断线程
- 必须记住保留锁，去保护程序中的重要部分，防止多步操作在执行的过程中中断，防止数据处于无效状态

协程
- 必须显式产出才能让程序的余下部分运行
- 无需保留锁，在多个线程之间同步操作，协程自身就会同步，因为在任意时刻只有一个协程运行
- 想要交出控制权时，可以使用 yield 或 yield from 把控制权交还给调度程序
- 按照定义，协程只能在暂停的 yield 处取消，因此可以处理 CancelledError 异常，执行清理操作

asyncio.Future 类的目的是与 yield from 一起使用，所以通常不需要使用以下方法
- my_future.add_done_callback(...)：协程是可以暂停和恢复的函数
- my_future.result()

对协程来说，获取 Task 对象有两种主要方式
- asyncio.async(coro\_or\_future, \*, loop=None)
- BaseEventLoop.create_task(coro)

asyncio 包中有多个函数会自动（内部使用的是 asyncio.async 函数）把参数指定的协程包装在 asyncio。Task 对象中
```python
import asyncio
def run_sync(coro_or_future):
    loop = asyncio.get_event_loop()
    return loop.run_until_complete(coro_or_future)

a = run_sync(some_coroutine())
```

在一个单线程程序中使用主循环依次激活队列里的协程。各个协程向前执行几步，然后把控制权让给主循环，主循环再激活队列里的下一个协程。

使用 asyncio 包时，异步代码中包含 asyncio 本身驱动的协程（即委派生成器），而生成器最终把职责委托给 asyncio 包或第三方库中的协程。

In [3]:
# flags_asyncio.py
import os
import time
import sys
import asyncio
import aiohttp


POP20_CC = ('CN IN US ID BR PK NG BD RU JP '
            'MX PH VN ET EG DE IR TR CD FR').split()

BASE_URL = 'http://flupy.org/data/flags'

DEST_DIR = 'downloads/'


def save_flag(img, filename):
    path = os.path.join(DEST_DIR, filename)
    with open(path, 'wb') as fp:
        fp.write(img)


async def get_flag(session, cc):  # 协程
    url = '{}/{cc}/{cc}.gif'.format(BASE_URL, cc=cc.lower())
    async with session.get(url) as resp:        # 阻塞操作
        return await resp.read()  # 读取响应内容


def show(text):
    print(text, end=' ')
    sys.stdout.flush()


async def download_one(session, cc):  # 协程
    image = await get_flag(session, cc)
    show(cc)
    save_flag(image, cc.lower() + '.gif')
    return cc


async def download_many(cc_list):
    async with aiohttp.ClientSession() as session:
        res = await asyncio.gather(                 # 下载国旗
            *[asyncio.create_task(download_one(session, cc))
                for cc in sorted(cc_list)])

    return len(res)

# def main():
#     t0 = time.time()
#     count = asyncio.run(download_many(POP20_CC))
#     elapsed = time.time() - t0
#     msg = '\n{} flags downloaded in {:.2f}s'
#     print(msg.format(count, elapsed))

async def main():  # <10>
    t0 = time.time()
    count = await download_many(POP20_CC)
    elapsed = time.time() - t0
    msg = '\n{} flags downloaded in {:.2f}s'
    print(msg.format(count, elapsed))
    

if __name__ == '__main__':
    await main()

EG FR JP ID DE NG RU TR IR CN VN BD US ET MX CD BR PK PH IN 
20 flags downloaded in 5.09s


阻塞型函数：执行硬盘或网络 I/O 操作的函数。

有两种方法能避免阻塞型调用终止整个应用程序的进程：
- 在单独的线程中运行各个阻塞型操作
- 把每个阻塞型操作转换成非阻塞的异步调用

为了降低内存的消耗，通常是由回调来实现异步调用。

把生成器当作协程是异步编程的另一种方式。各个暂停的协程是要消耗内存，但是比线程消耗的内存数量级小。

Semaphore 对象维护着一个内部计数器，若在对象上调用 .acquire() 协程方法，计数器则递减；若在对象上调用 .release() 协程方法，计数器则递增。计数器的初始值在实例化 Semaphore 时设定。
- 如果计数器大于零，调用 .acquire() 不会阻塞
- 如果计时器为零，调用 .acquire() 会阻塞调用这个方法的协程，直到其他协程在同一个 Semaphore 对象上调用 .release() 方法，让计数器递减

In [ ]:
# flags2_asyncio.py
import asyncio
import collections

import aiohttp
from aiohttp import web
import tqdm

from flags2_common import process_args, HTTPStatus, Result, save_flag

# default set low to avoid errors from remote site, such as
# 503 - Service Temporarily Unavailable
DEFAULT_CONCUR_REQ = 5
MAX_CONCUR_REQ = 1000


class FetchError(Exception):  # 自定义异常
    def __init__(self, country_code):
        self.country_code = country_code


async def get_flag(session, base_url, cc):
    url = '{}/{cc}/{cc}.gif'.format(base_url, cc=cc.lower())
    async with session.get(url) as resp:
        if resp.status == 200:
            return await resp.read()
        elif resp.status == 404:
            raise web.HTTPNotFound()
        else:
            raise aiohttp.HttpProcessingError(
                code=resp.status, message=resp.reason,
                headers=resp.headers)


async def download_one(session, cc, base_url, semaphore, verbose):  # Semaphore 类是同步装置，用于限制并发请求数量
    try:
        async with semaphore:  # 当成上下文管理器使用，防止阻塞整个系统：如果 semaphore 计数器的值是所允许的最大值，只有这个协程阻塞
            image = await get_flag(session, base_url, cc)  # semaphore 计数器递减，解除阻塞可能在等待同一个 semaphore 对象的其他协程实例
    except web.HTTPNotFound:  # 没找到国旗，设置相应状态
        status = HTTPStatus.not_found
        msg = 'not found'
    except Exception as exc:
        raise FetchError(cc) from exc  # 链接异常
    else:
        save_flag(image, cc.lower() + '.gif')  # 保存文件，阻塞
        status = HTTPStatus.ok
        msg = 'OK'

    if verbose and msg:
        print(cc, msg)

    return Result(status, cc)


async def downloader_coro(cc_list, base_url, verbose, concur_req):  # <1>
    counter = collections.Counter()
    semaphore = asyncio.Semaphore(concur_req)  # <2>
    async with aiohttp.ClientSession() as session:  # <8>
        to_do = [download_one(session, cc, base_url, semaphore, verbose)
                for cc in sorted(cc_list)]  # <3>

        to_do_iter = asyncio.as_completed(to_do)  # <4>
        if not verbose:
            to_do_iter = tqdm.tqdm(to_do_iter, total=len(cc_list))  # <5>
        for future in to_do_iter:  # <6>
            try:
                res = await future  # <7>
            except FetchError as exc:  # <8>
                country_code = exc.country_code  # <9>
                try:
                    error_msg = exc.__cause__.args[0]  # <10>
                except IndexError:
                    error_msg = exc.__cause__.__class__.__name__  # <11>
                if verbose and error_msg:
                    msg = '*** Error for {}: {}'
                    print(msg.format(country_code, error_msg))
                status = HTTPStatus.error
            else:
                status = res.status

            counter[status] += 1  # <12>

    return counter  # <13>


def download_many(cc_list, base_url, verbose, concur_req):
    loop = asyncio.get_event_loop()
    coro = downloader_coro(cc_list, base_url, verbose, concur_req)
    counts = loop.run_until_complete(coro)  # <14>
    loop.close()  # <15>

    return counts


def main(download_many, default_concur_req, max_concur_req):
    args, cc_list = process_args(default_concur_req)
    actual_req = min(args.max_req, max_concur_req, len(cc_list))
    initial_report(cc_list, actual_req, args.server)
    base_url = SERVERS[args.server]
    t0 = time.time()
    counter = download_many(cc_list, base_url, args.verbose, actual_req)
    assert sum(counter.values()) == len(cc_list), \
        'some downloads are unaccounted for'
    final_report(cc_list, counter, t0)


if __name__ == '__main__':
    main(download_many, DEFAULT_CONCUR_REQ, MAX_CONCUR_REQ)

回调地狱：如果一个操作需要依赖之前操作的结果，那就得嵌套回调

```python
def stage1(response1):
    request2 = step1(response1)
    api_call2(request2, stage2)
    
def stage2(response2):
    requst3 = step2(response2)
    api_call3(request3, stage3)

def stage3(response3):
    step3(response3)

api_call1(request1, stage1)
```

使用协程做异步编程，无需使用回调
- 后续处理便于使用前一步的结果
- 提供了上下文，能通过异常来报告错误

```python
async def three_stages(request1):
    reponse1 = await api_call1(request1)
    # 第一步
    request2 = step1(response1)
    response2 = await api_call1(request1)
    # 第二步
    request3 = step2(response2)
    response3 = await api_call1(request1)
    # 第三步
    step3(response3)

await three_stages(request1)
```

StreamWriter.write 是普通的函数，我们假定它大多数时候都不会阻塞，因为它把数据写入缓冲；而刷新缓冲并真正执行 I/O 操作的 StreamWriter.drain 是协程，StreamReader.readline 也是协程

In [ ]:
# tcp_charfinder
import sys
import asyncio

from charfinder import UnicodeNameIndex  # 用于构建名称索引，提供查询方法

CRLF = b'\r\n'
PROMPT = b'?> '

index = UnicodeNameIndex()  # 使用 charfinder_index.pickle 文件（如果有的话）

async def handle_queries(reader, writer):  # 协程，参数：asyncio.StreamReader 对象、asyncio.StreamWriter 对象
    while True:  # <4>
        writer.write(PROMPT)  # can't await!  # 普通函数
        await writer.drain()  # must await!  # 协程，刷新缓冲
        data = await reader.readline()  # 协程
        try:
            query = data.decode().strip()
        except UnicodeDecodeError:  # 假装发送的是空字符
            query = '\x00'
        client = writer.get_extra_info('peername')  # 返回与套接字连接的远程地址
        print('Received from {}: {!r}'.format(client, query))  # 在服务器的控制台中查询记录
        if query:
            if ord(query[:1]) < 32:  # 空字符或控制字符
                break
            lines = list(index.find_description_strs(query)) # 返回生成器，产出包含 Unicode 码位、真正的字符、字符名称的字符串
            if lines:
                writer.writelines(line.encode() + CRLF for line in lines) # 默认 UTF-8 编码
            writer.write(index.status(query, len(lines)).encode() + CRLF) # 输出状态

            await writer.drain()  # 刷新缓冲
            print('Sent {} results'.format(len(lines)))  # 记录响应

    print('Close the client socket')  # 记录会话结束
    writer.close()  # 关闭 StreamWriter 流


# 创建并销毁事件循环和套接字服务器
async def main(address='127.0.0.1', port=2323):
    port = int(port)
    server = await asyncio.start_server(handle_queries, address, port) # 返回 asyncio.Server 实例（TCP 套接字服务器）

    host = server.sockets[0].getsockname()  # 获取服务器的第一个套接字的地址和端口
    print('Serving on {}. Hit CTRL-C to stop.'.format(host))

    async with server:
        await server.serve_forever()


# if __name__ == '__main__':
#     asyncio.run(main(*sys.argv[1:]))  # <5>

if __name__ == '__main__':
    await main()

asyncio.start_server 函数和 loop.create_server 方法都是协程，返回的结果都是 asyncio.Server 对象

只有驱动协程，协程才能做事。驱动 asyncio.coroutine 装饰的协程有两种方法，要么使用 yield from / await ，要么传给 asyncio 包中某个参数为协程或 future 的函数

In [ ]:
# http_charfinder.py
import sys
import asyncio
from aiohttp import web

from charfinder import UnicodeNameIndex

TEMPLATE_NAME = 'http_charfinder.html'
CONTENT_TYPE = 'text/html; charset=UTF-8'
SAMPLE_WORDS = ('bismillah chess cat circled Malayalam digit'
                ' Roman face Ethiopic black mark symbol dot'
                ' operator Braille hexagram').split()

ROW_TPL = '<tr><td>{code_str}</td><th>{char}</th><td>{name}</td></tr>'
LINK_TPL = '<a href="/?query={0}" title="find &quot;{0}&quot;">{0}</a>'
LINKS_HTML = ', '.join(LINK_TPL.format(word) for word in
                       sorted(SAMPLE_WORDS, key=str.upper))


index = UnicodeNameIndex()
with open(TEMPLATE_NAME) as tpl:
    template = tpl.read()
template = template.replace('{links}', LINKS_HTML)


def home(request):  # 路由处理函数
    query = request.GET.get('query', '').strip()  # 去掉首尾空白
    print('Query: {!r}'.format(query))  # 记录查询
    if query:  # <4>
        descriptions = list(index.find_descriptions(query))
        res = '\n'.join(ROW_TPL.format(**descr._asdict())
                        for descr in descriptions)
        msg = index.status(query, len(descriptions))
    else:
        descriptions = []
        res = ''
        msg = 'Enter words describing characters.'

    html = template.format(query=query, result=res,  # 渲染 HTML 页面
                           message=msg)
    print('Sending {} results'.format(len(descriptions)))  # 记录响应
    return web.Response(content_type=CONTENT_TYPE, text=html) # 返回响应


@asyncio.coroutine
def init(loop, address, port):  # 协程
    app = web.Application(loop=loop)  # Web 应用
    app.router.add_route('GET', '/', home)  # 路由到 home 函数
    handler = app.make_handler()  # 根据 app 对象设置的路由处理 HTTP 请求
    server = yield from loop.create_server(handler,
                                           address, port)  # 创建服务器
    return server.sockets[0].getsockname()  # 返回服务器套接字的地址和端口

def main(address="127.0.0.1", port=2345):
    loop = asyncio.get_event_loop()
    host = loop.run_until_complete(init(loop, address, port))  # 启动服务器
    print('Serving on {}. Hit CTRL-C to stop.'.format(host))
    try:
        loop.run_forever()  # 阻塞
    except KeyboardInterrupt:  # CTRL+C pressed
        pass
    print('Server shutting down.')
    loop.close()  # 关闭事件循环


if __name__ == '__main__':
    main(*sys.argv[1:])

避免响应时间太长的方法是实现分页：首次至多返回（比如）200 行，用户点击链接或滚动页面时再获取更多结果。

实现分批发送结果所需的大多数代码都在浏览器这一端，智能的异步客户端能更好地使用服务器资源。

异步系统能避免用户级线程的开销，这是它能比多线程系统管理更多并发连接的主要原因。

至尊循环（the One Loop）：

    至尊循环驭众生，至尊循环寻众生，
    
    至尊循环引众生，普照众生欣欣荣。
    
梗：《魔戒》